## GMM from scratch

Build a simple implementation of GMM using the expectation-maximization algorithm and numpy operations. Run model on built in IRIS dataset. 

Based on the article: https://towardsdatascience.com/gaussian-mixture-models-explained-6986aaf5a95

### Concepts

1. Covariance matrix: https://en.wikipedia.org/wiki/Covariance_matrix 
2. Guassian distribution: https://en.wikipedia.org/wiki/Gaussian_function
3. Determinant: https://en.wikipedia.org/wiki/Determinant
4. Eigenvectors: https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors

### GMM

1. User Guide: https://scikit-learn.org/stable/modules/mixture.html
2. Class API: https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html
3. Model selection: https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_selection.html
4. Expectation-maximization: https://en.wikipedia.org/wiki/Expectation

### Parameters

1. log-likelihood = probability of parameters for known x outcome
2. $\gamma(z_{nk})$ = expectation probability
3. $\pi_k$ = gaussian mixing component 
4. $\mu_k$ = component mean
5. $\Sigma_k$ = component covariance

In [37]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [108]:
# build dataset
iris = datasets.load_iris()
X = iris.data
X[0].shape

(4,)

In [109]:
# TODO: move to module

def gdf(X, mu, cov):
    # get number of elements in instance 
    n = X.shape[1]
    # substract features from mean, transpose for eventual dot product
    diff = (X - mu).T
    # compute left side, return a single float
    left = 1/((2*np.pi)**(n/2)*np.linalg.det(cov)**0.5)
    # compute right side, extract values on diagnonal of matrix, return row vector
    right = np.diagonal(np.exp(-0.5*np.dot(np.dot(diff.T, np.linalg.inv(cov)), diff)))
    # multiply left scaler by right vector, reshape to column vector and return
    gdf = (left*right).reshape(-1,1)

    return gdf


def init_clusters(X, n_clusters):
    clusters = []
    # fit iris data to kmeans
    kmeans = KMeans().fit(X)
    # extract 4 cluster means
    mu_k = kmeans.cluster_centers_
    
    # build dict of init params for each cluster 
    for i in range(n_clusters):
        # append param key value pairs to list
        clusters.append({
            # inverse of n_cluster
            'pi_k': 1.0 / n_clusters,
            # means for a given cluster
            'mu_k': mu_k[i],
            # n x n identity matrix from dimensions of iris data
            'cov_k': np.identity(X.shape[1], dtype=np.float64)
        })
    
    return clusters


def expectation_step(X, clusters):
    # make array to hold totals
    totals = np.zeros((X.shape[0], 1), dtype=np.float64)
    
    for cluster in clusters:
        # grab param values
        pi_k = cluster['pi_k']
        mu_k = cluster['mu_k']
        cov_k = cluster['cov_k']
        # compute numerator of gamma and return a column vector of length X
        gamma_nk = (pi_k * gaussian(X, mu_k, cov_k)).astype(np.float64)

        # compute denominator of gamma as sum of numerator terms and return a column vector of length X
        for i in range(X.shape[0]):
            totals[i] += gamma_nk[i]
        
        # assign to keys in dictionary
        cluster['gamma_nk'] = gamma_nk
        cluster['totals'] = totals
    
    # complete computation of gamma_nk, not the biggest fan of this method
    for cluster in clusters:
        cluster['gamma_nk'] /= cluster['totals']


def maximization_step(X, clusters):
    # get N equal to shape of X
    N = float(X.shape[0])
  
    for cluster in clusters:
        # grab gamma column vector
        gamma_nk = cluster['gamma_nk']
        # make n x n cov matrix from iris dimensions 
        cov_k = np.zeros((X.shape[1], X.shape[1]))
        # get sum of gamma
        N_k = np.sum(gamma_nk, axis=0)
        # compute updated pi_k
        pi_k = N_k / N
        # compute updated mu_k
        mu_k = np.sum(gamma_nk * X, axis=0) / N_k
        
        # compute updated cov_k
        for j in range(X.shape[0]):
            diff = (X[j] - mu_k).reshape(-1, 1)
            cov_k += gamma_nk[j] * np.dot(diff, diff.T)
        
        # complete cov_k
        cov_k /= N_k
        
        # update parameters in dict
        cluster['pi_k'] = pi_k
        cluster['mu_k'] = mu_k
        cluster['cov_k'] = cov_k


def get_likelihood(X, clusters):
    # compute sample likelihoods by accessing totals key in each cluster
    return np.log(np.array([cluster['totals'] for cluster in clusters]))


# nest init, expectation, and maximization in a training function
# cluster list of dictionaries are avaialbe in local scope of function
def train_gmm(X, n_clusters, n_epochs):
    # init clusters 
    clusters = init_clusters(X, n_clusters)

    for i in range(n_epochs):
        # run expectation over each epoch
        expectation_step(X, clusters)
        # run maximization over each epoch
        maximization_step(X, clusters)
        # compute likelihoods 
        sample_likelihoods = get_likelihood(X, clusters)
        
    return clusters, sample_likelihoods

In [111]:
# clusters to search for and epochs to run EM over
n_clusters = 3
n_epochs = 50

# run scratch routine and assign values to variables
clusters, sample_likelihoods = train_gmm(X, n_clusters, n_epochs)

# construct and fit sklearn gmm to iris data
gmm = GaussianMixture(n_components=n_clusters, max_iter=n_epochs).fit(X)
# grab model log-lilelihood
gmm_scores = gmm.score_samples(X)


In [112]:
# comparison between scratch gmm and sklearn gmm

# cluster means for scratch and sklearn
print('Means by sklearn:\n', gmm.means_)
print('Means by our implementation:\n', np.array([cluster['mu_k'].tolist() for cluster in clusters]))

# model likelihood scores for scratch and sklearn
print('Scores by sklearn:\n', gmm_scores[0:20])
print('Scores by our implementation:\n', sample_likelihoods.reshape(-1)[0:20])

Means by sklearn:
 [[5.006      3.428      1.462      0.246     ]
 [5.91697517 2.77803998 4.20523542 1.29841561]
 [6.54632887 2.94943079 5.4834877  1.98716063]]
Means by our implementation:
 [[5.91496959 2.77784365 4.20155323 1.29696685]
 [5.006      3.428      1.462      0.246     ]
 [6.54454865 2.94866115 5.47955343 1.98460495]]
Scores by sklearn:
 [ 1.57050082  0.73787138  1.14436656  0.92913238  1.411028   -0.09451903
  0.05266884  1.62442195  0.27082378  0.16706624  0.83489877  0.77168582
  0.29597841 -1.79224582 -3.41557928 -2.10529279 -1.12995447  1.47503579
 -0.84612536  0.97699215]
Scores by our implementation:
 [ 1.57057947  0.73793642  1.14444614  0.92920539  1.41110417 -0.09448868
  0.05268031  1.62449505  0.27090462  0.16702226  0.83494742  0.77171947
  0.29597776 -1.79222469 -3.41562626 -2.1052825  -1.1300608   1.47509939
 -0.84608424  0.9770596 ]
